<a href="https://colab.research.google.com/github/SherifNabil-Eng/AzureCognitivePoCs/blob/main/TestFormRecHandWritting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install azure-ai-formrecognizer==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 1.6 MB/s eta 0:00:00


In [ ]:
# import libraries
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount('/content/drive', force_remount=force_remount)

import os
import sys
import numpy as np
import pandas as pd
from datetime import date

from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

os.chdir("/content/gdrive/My Drive/Colab Notebooks/Tests")


Mounted at /content/gdrive


In [ ]:
def format_bounding_region(bounding_regions):
    if not bounding_regions:
        return "N/A"
    return ", ".join("Page #{}: {}".format(region.page_number, format_polygon(region.polygon)) for region in bounding_regions)

def format_polygon(polygon):
    if not polygon:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in polygon])

In [ ]:
def make_file_path(document_file_name):
  
  currroot=os.getcwd()
  print("File location using os.getcwd():", os.getcwd())
  document_path_temp = os.path.join(currroot, document_file_name)

  return document_path_temp

In [ ]:
def analyze_doc (document):


  endpoint = ""
  key = ""
  lines=[]

  document_analysis_client = DocumentAnalysisClient(
      endpoint=endpoint, credential=AzureKeyCredential(key)
      )

  with open(document, "rb") as f:
    poller = document_analysis_client.begin_analyze_document(
        "prebuilt-read", document=f
        )
    result = poller.result()

  print("----Languages detected in the document----")
    
  for language in result.languages:
        print("Language code: '{}' with confidence {}".format(language.locale, language.confidence))

  for page in result.pages:
    
    print("----Analyzing document from page #{}----".format(page.page_number))
    print( "Page has width: {} and height: {}, measured with unit: {}".format(
        page.width, page.height, page.unit
        )
    )

    for line_idx, line in enumerate(page.lines):
      words = line.get_words()
      print(
          
          "...Line # {} has {} words and text '{}' within bounding polygon '{}'".format(
              line_idx,
              len(words),
              line.content,
              format_polygon(line.polygon),
              )
          )
      lines.append (line.content)


  
  
  
  return lines

In [ ]:
if __name__ == "__main__":

  documents =["FormRecImgs/bday_Farah.jpg","FormRecImgs/Marriage_sherif_yasmine.jpg","FormRecImgs/hand_written_receipt.jpg"]
  
  document_lines=[]
 

  try:
    for i in range (0,len(documents)):
      document_path= make_file_path(documents[i])
      #document_content = analyze_doc(document_path)
      document_lines.append(analyze_doc(document_path))
  except HttpResponseError as error:
    
    # Examples of how to check an HttpResponseError
    # Check by error code:
    if error.error is not None:

      if error.error.code == "InvalidRequest":

        print(f"Received an invalid request error: {error.error}")
        sys.exit(1)
      if error.error.code == "InvalidImage":

        print(f"Received an invalid image error: {error.error}")
        sys.exit(1)
        # If the inner error is None and then it is possible to check the message to get more information:
    filter_msg = ["Generic error", "Timeout", "Invalid request", "InvalidImage"]
    if any(example_error.casefold() in error.message.casefold() for example_error in filter_msg):
      print(f"Uh-oh! Something unexpected happened: {error}")
      sys.exit(1)
        # Print the full error content:
    print(f"Full HttpResponseError: {error}")

File location using os.getcwd(): /content/gdrive/MyDrive/Colab Notebooks/Tests
----Languages detected in the document----
Language code: 'ar' with confidence 1.0
Language code: 'en' with confidence 0.6
Language code: 'otq' with confidence 0.8
Language code: 'fa' with confidence 1.0
Language code: 'en' with confidence 0.7
Language code: 'ht' with confidence 0.6
Language code: 'ar' with confidence 0.8
Language code: 'otq' with confidence 0.4
Language code: 'en' with confidence 0.99
Language code: 'ar' with confidence 0.7
Language code: 'ar' with confidence 0.95
Language code: 'ar' with confidence 0.9
----Analyzing document from page #1----
Page has width: 697.0 and height: 1024.0, measured with unit: pixel
...Line # 0 has 5 words and text 'أهلة القاهرة سحا مدنى الخزانة' within bounding polygon '[78.0, 150.0], [193.0, 168.0], [188.0, 202.0], [75.0, 190.0]'
...Line # 1 has 3 words and text 'صورة قيد الميلاد:٢' within bounding polygon '[216.0, 172.0], [418.0, 166.0], [419.0, 207.0], [216.0,

In [ ]:
for i in range (0,len(documents)):
  index_slash=documents[i].find ("/")
  index_dot=documents[i].find (".")

  file_sub_name = documents[i][index_slash+1:index_dot]

  current_date = today = str (date.today())

  extracted_file_name = file_sub_name+"FormRecHandWritting"+current_date+".txt"

  file_extracted_handle = open(extracted_file_name,"a")

  for j in range (0,len(document_lines[i])):
    file_extracted_handle.write(document_lines[i][j]+"\n")
  


file_extracted_handle.close()